In [1]:
import shutil
import os
import torchaudio
import librosa
import torch
import pickle as pkl
import audiofile
from soundfile import LibsndfileError
from speechbrain.inference.classifiers import EncoderClassifier

# Clear the model cache
cache_dir = "C:/Users/Adria/.cache/huggingface/hub/models--speechbrain--lang-id-voxlingua107-ecapa"
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

# Also remove the tmp directory
if os.path.exists("tmp"):
    shutil.rmtree("tmp")

# Then try loading the model again
from speechbrain.inference.classifiers import EncoderClassifier
language_id = EncoderClassifier.from_hparams(source="speechbrain/lang-id-voxlingua107-ecapa", savedir="tmp")


c:\Users\Adria\dev\blindwikiapp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Adria\dev\blindwikiapp\.venv\Lib\site-packages\speechbrain\utils\fetching.py:151: UserWarning: Using SYMLINK strategy on Windows for fetching potentially requires elevated privileges and is not recommended. See `LocalStrategy` documentation.
  warnings.warn(
c:\Users\Adria\dev\blindwikiapp\.venv\Lib\site-packages\speechbrain\utils\autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
C:\Users\Adria\AppData\Local\Programs\Python\Python312\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.infe

In [2]:
 # to be run on top of the 'uploads' directory containing the audio files
path = os.path.join(os.getcwd(), "uploads")
files = os.listdir(path)
supported_audio_formats = ["wav","mp3"]
files = [file for file in files if file[-3:] in supported_audio_formats]

In [3]:
path, files

('c:\\Users\\Adria\\dev\\blindwikiapp\\notebooks\\uploads',
 ['acoruna_barco_m67780_a84524_audio_converted.mp3',
  'barcelona_Ovvero_m68255_a84999_audio_converted.mp3',
  'berlin_Dirk_m67384_a84128_audio_converted.mp3',
  'cuenca_Amalia_m69879_a86623_audio_converted.mp3',
  'elbarcelonC3A8s_Martagosa_m71623_a88367_audio_converted.mp3'])

In [4]:
file_ends = set()
for file in files:
    file_ends.add(file[-3:])
file_ends

{'mp3'}

In [5]:
# Use the path variable you already defined
file_path = os.path.join(path, "barcelona_Ovvero_m68255_a84999_audio_converted.mp3")
os.path.exists(file_path)
signal, sampling_rate = audiofile.read(file_path)
signal

array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -9.4561778e-07, -6.0664206e-07, -3.3898604e-07],
      shape=(254016,), dtype=float32)

In [7]:
# Replace the last cell in language_tagger.ipynb
predictions_dict = {}  # Change from list to dictionary
corrupted_files = []
predictions_file = "predictions_dict.pkl"
i = 0
for file in files:
    #print(file)
    i+=1
    try:
        signal, sr = audiofile.read(os.path.join(path, file))
    except LibsndfileError as e:
        print("Corrupt file", file)
        print(e)
        corrupted_files.append(file)
        continue
    prediction = language_id.classify_batch(torch.tensor(signal))
    language = prediction[3][0].split(":")[1].strip()
    predictions_dict[file] = language  # Save as dictionary with filename as key
    if i % 100 == 0:
        print(f"Iteration {i} completed reaching update...")
        with open(os.path.join("data", predictions_file), "wb") as f:  # Save to data directory
            pkl.dump(predictions_dict, f)
print(predictions_dict)
with open(os.path.join("data", predictions_file), "wb") as f:  # Save to data directory
    pkl.dump(predictions_dict, f)

{'acoruna_barco_m67780_a84524_audio_converted.mp3': 'Galician', 'barcelona_Ovvero_m68255_a84999_audio_converted.mp3': 'Bosnian', 'berlin_Dirk_m67384_a84128_audio_converted.mp3': 'German', 'cuenca_Amalia_m69879_a86623_audio_converted.mp3': 'Spanish', 'elbarcelonC3A8s_Martagosa_m71623_a88367_audio_converted.mp3': 'Catalan'}
